# Load Util File with TARE Model Functions

In [1]:
import os

# Measure Package 0: Baseline
menu_mp = 0
input_mp = 'baseline'

# Get the current working directory of the project
project_root = os.path.abspath(os.getcwd())
print(f"Project root directory: {project_root}")

# Relative path to the file from the project root
relative_path = r"tare_model_functions_v2.3.ipynb"

# Construct the absolute path to the file
file_path = os.path.join(project_root, relative_path)
print(f"File path: {file_path}")

# Run the notebook and import variables
if os.path.exists(relative_path):
    get_ipython().run_line_magic('run', f'-i "{relative_path}"')
    print("Loaded All TARE Model Functions")
else:
    print(f"File not found: {relative_path}")

Project root directory: c:\Users\14128\Research\cmu-tare-model
File path: c:\Users\14128\Research\cmu-tare-model\tare_model_functions_v2.3.ipynb
Loaded All TARE Model Functions


In [2]:
# Storing Result Outputs in output_results folder
relative_path = r"output_results"
output_folder_path = os.path.join(project_root, relative_path)
print(f"Result outputs will be exported here: {output_folder_path}")

Result outputs will be exported here: c:\Users\14128\Research\cmu-tare-model\output_results


# Baseline: 

## Simulate Residential Energy Consumption
- Filter EUSS Data: Only occupied units and Single Family Homes



In [3]:
# The ``inline`` flag will use the appropriate backend to make figures appear inline in the notebook.  
%matplotlib inline

import pandas as pd
import numpy as np

# `plt` is an alias for the `matplotlib.pyplot` module
import matplotlib.pyplot as plt

# import seaborn library (wrapper of matplotlib)
import seaborn as sns
sns.set(style="darkgrid")

# For regex, import re
import re

from datetime import datetime

# Get the current datetime
# Start the timer
start_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [4]:
print("""
-------------------------------------------------------------------------------------------------------
Welcome to the Trade-off Analysis of residential Retrofits for energy Equity Tool (TARE Model)
Let's start by reading the data from the NREL EUSS Database.

Make sure that the zipped folders stay organized as they are once unzipped.
If changes are made to the file path, then the program will not run properly.
-------------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------------
BASELINE (Measure Package 0)
-------------------------------------------------------------------------------------------------------
""")

# Measure Package 0: Baseline
menu_mp = 0
input_mp = 'baseline'

filename = "baseline_metadata_and_annual_results.csv"
relative_path = os.path.join(r"euss_data\resstock_amy2018_release_1.1\state", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

print("""
-------------------------------------------------------------------------------------------------------
Data Filters: Only occupied units and Single Family Homes
-------------------------------------------------------------------------------------------------------
""")

# Fix DtypeWarning error in columns:
# 'in.neighbors', 'in.geometry_stories_low_rise', 'in.iso_rto_region', 'in.pv_orientation', 'in.pv_system_size'
columns_to_string = {11: str, 61: str, 121: str, 103: str, 128: str, 129: str}
df_euss_am_baseline = pd.read_csv(file_path, dtype=columns_to_string)
occupancy_filter = df_euss_am_baseline['in.vacancy_status'] == 'Occupied'
df_euss_am_baseline = df_euss_am_baseline.loc[occupancy_filter]

# Filter for single family home building type
house_type_list = ['Single-Family Attached', 'Single-Family Detached']
house_type_filter = df_euss_am_baseline['in.geometry_building_type_recs'].isin(house_type_list)
df_euss_am_baseline = df_euss_am_baseline.loc[house_type_filter]
# df_euss_am_baseline


-------------------------------------------------------------------------------------------------------
Welcome to the Trade-off Analysis of residential Retrofits for energy Equity Tool (TARE Model)
Let's start by reading the data from the NREL EUSS Database.

Make sure that the zipped folders stay organized as they are once unzipped.
If changes are made to the file path, then the program will not run properly.
-------------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------------
BASELINE (Measure Package 0)
-------------------------------------------------------------------------------------------------------

Retrieved data for filename: baseline_metadata_and_annual_results.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\euss_data\resstock_amy2018_release_1.1\state\baseline_metadata_and_annual_results.csv

------------------------

In [5]:
# # Make a copy of the dataframe
# df_euss_am_baseline = df_euss_am_baseline.copy()

# Choose between national or sub-national level analysis
menu_state = get_menu_choice(menu_prompt, {'N', 'Y'})   # This code is only run in baseline

# National Level 
if menu_state == 'N':
    print("You chose to analyze all of the United States.")
    input_state = 'National'

# Filter down to state or city
else:
    input_state = get_state_choice(df_euss_am_baseline)
    print(f"You chose to filter for: {input_state}")
    state_filter = df_euss_am_baseline['in.state'].eq(input_state)
    df_euss_am_baseline = df_euss_am_baseline.loc[state_filter]

    print(city_prompt)
    print(df_euss_am_baseline['in.city'].value_counts())

    menu_city = get_menu_choice(city_menu_prompt, {'N', 'Y'})

    # Filter for the entire selected state
    if menu_city == 'N':
        print(f"You chose to analyze all of state: {input_state}")
        
    # Filter to a city within the selected state
    else:
        input_cityFilter = get_city_choice(df_euss_am_baseline, input_state)
        print(f"You chose to filter for: {input_state}, {input_cityFilter}")
        city_filter = df_euss_am_baseline['in.city'].eq(f"{input_state}, {input_cityFilter}")
        df_euss_am_baseline = df_euss_am_baseline.loc[city_filter]

# Display the filtered dataframe
df_euss_am_baseline

You chose to analyze all of the United States.


,bldg_id,upgrade,weight,applicability,in.sqft,in.ahs_region,in.ashrae_iecc_climate_zone_2004,in.ashrae_iecc_climate_zone_2004_2_a_split,in.bathroom_spot_vent_hour,in.bedrooms,...,out.emissions.natural_gas.lrmer_low_re_cost_25_2025_start.co2e_kg,out.emissions.propane.lrmer_low_re_cost_25_2025_start.co2e_kg,out.emissions.electricity.lrmer_mid_case_15_2025_start.co2e_kg,out.emissions.fuel_oil.lrmer_mid_case_15_2025_start.co2e_kg,out.emissions.natural_gas.lrmer_mid_case_15_2025_start.co2e_kg,out.emissions.propane.lrmer_mid_case_15_2025_start.co2e_kg,out.emissions.all_fuels.lrmer_95_decarb_by_2035_15_2025_start.co2e_kg,out.emissions.all_fuels.lrmer_low_re_cost_15_2025_start.co2e_kg,out.emissions.all_fuels.lrmer_low_re_cost_25_2025_start.co2e_kg,out.emissions.all_fuels.lrmer_mid_case_15_2025_start.co2e_kg
2,239,0,242.131013,True,1690.0,Non-CBSA East South Central,3A,3A,Hour20,3,...,215.943534,0.000000,8773.384074,0.0,215.943534,0.000000,3565.038262,6416.193347,5755.373221,8989.327608
3,273,0,242.131013,True,1690.0,Non-CBSA East South Central,3A,3A,Hour12,3,...,0.000000,0.000000,11296.731129,0.0,0.000000,0.000000,4339.903757,8193.678510,7254.665194,11296.731129
4,307,0,242.131013,True,1220.0,Non-CBSA East South Central,3A,3A,Hour0,4,...,0.000000,0.000000,8750.011820,0.0,0.000000,0.000000,3345.187937,6249.625611,5587.946834,8750.011820
5,409,0,242.131013,True,1220.0,Non-CBSA East South Central,3A,3A,Hour20,2,...,1642.477930,0.000000,5725.103641,0.0,1642.477930,0.000000,3784.993820,5489.549041,5164.458936,7367.581571
7,517,0,242.131013,True,1220.0,Non-CBSA East South Central,3A,3A,Hour1,3,...,0.000000,0.000000,8932.439414,0.0,0.000000,0.000000,3441.414837,6415.303853,5721.073473,8932.439414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548907,548226,0,242.131013,True,2176.0,Non-CBSA Mountain,6B,6B,Hour3,4,...,15324.356044,0.000000,2313.298407,0.0,15324.356044,0.000000,16802.023908,16545.095582,16472.920871,17637.654451
548908,548228,0,242.131013,True,1690.0,Non-CBSA Mountain,6B,6B,Hour6,4,...,8192.601682,0.000000,1889.439924,0.0,8192.601682,0.000000,9394.122057,9195.903552,9129.578822,10082.041606
548910,548417,0,242.131013,True,885.0,Non-CBSA Mountain,6B,6B,Hour18,2,...,5212.758359,0.000000,2112.907195,0.0,5212.758359,0.000000,6546.826589,6318.763521,6253.986448,7325.665554
548914,549740,0,242.131013,True,1220.0,Non-CBSA Mountain,7B,7B,Hour4,2,...,0.000000,268.627834,11423.104685,0.0,0.000000,268.627834,7173.561517,5931.433285,4609.821155,11691.732519


## Baseline Energy Consumption
### Factors to Project Future Energy Consumption Using EIA Heating Degree Day (HDD) Forecasted Data

In [6]:
filename = 'eia_cdd_hdd_projections.xlsx'
relative_path = os.path.join(r"weather_data", filename)
file_path = os.path.join(project_root, relative_path)
df_hdd_projection_factors = pd.read_excel(io=file_path, sheet_name='hdd_projection_factors')

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

# Convert the factors dataframe into a lookup dictionary
hdd_factor_lookup = df_hdd_projection_factors.set_index(['census_division']).to_dict('index')
hdd_factor_lookup

Retrieved data for filename: eia_cdd_hdd_projections.xlsx
Located at filepath: c:\Users\14128\Research\cmu-tare-model\weather_data\eia_cdd_hdd_projections.xlsx


{'National': {'historic_data': 'Heating Degree days: U.S.',
  'projections_data': 'Residential: Heating Degree Days: United States: Reference case',
  'units': 'HDD',
  2018: 1,
  2019: 1.0060591936611512,
  2020: 0.912141691913307,
  2021: 0.9168026101141925,
  2022: 0.986859419715684,
  2023: 0.9896571083663482,
  2024: 0.9266152311815428,
  2025: 0.9222047690515032,
  2026: 0.9178609890468422,
  2027: 0.9135426984385924,
  2028: 0.9091717790724773,
  2029: 0.9048058664646935,
  2030: 0.9004451883010953,
  2031: 0.8961055616406431,
  2032: 0.8917795900722443,
  2033: 0.8874326238639012,
  2034: 0.8831163243999067,
  2035: 0.8787744789093451,
  2036: 0.8743959354462829,
  2037: 0.8701421647634584,
  2038: 0.8658367895595432,
  2039: 0.8614605788860406,
  2040: 0.8571702241901654,
  2041: 0.8529132104404568,
  2042: 0.8486123299930086,
  2043: 0.8443018340713121,
  2044: 0.8399903710090888,
  2045: 0.8356955213237007,
  2046: 0.8314379955721277,
  2047: 0.8272099988347704,
  2048: 0.82

In [7]:
print("""
-------------------------------------------------------------------------------------------------------
Baseline Consumption:
-------------------------------------------------------------------------------------------------------
""")

# df_baseline_enduse(df_baseline, df_enduse, category, fuel_filter='Yes', tech_filter='Yes')
df_euss_am_baseline_home = df_enduse_refactored(df_baseline = df_euss_am_baseline,
                                                fuel_filter = 'Yes',
                                                tech_filter = 'Yes')

# Project Future Energy Consumption
df_euss_am_baseline_home = project_future_consumption(df=df_euss_am_baseline_home, hdd_factor_lookup=hdd_factor_lookup)
df_euss_am_baseline_home


-------------------------------------------------------------------------------------------------------
Baseline Consumption:
-------------------------------------------------------------------------------------------------------

Processing column: in.clothes_dryer
Initial data types: object
Data types after processing: object
Processing column: in.cooking_range
Initial data types: object
Data types after processing: object
331531 rows remain after applying total heating consumption calculation
Filtered for the following fuels: ['Natural Gas', 'Electricity', 'Propane', 'Fuel Oil']
321357 rows remain after applying heating fuel filter
Filtered for the following Heating technologies: ['Electricity ASHP', 'Electricity Baseboard', 'Electricity Electric Boiler', 'Electricity Electric Furnace', 'Fuel Oil Fuel Boiler', 'Fuel Oil Fuel Furnace', 'Natural Gas Fuel Boiler', 'Natural Gas Fuel Furnace', 'Propane Fuel Boiler', 'Propane Fuel Furnace']
291558 rows remain after applying heating techn

,bldg_id,square_footage,census_region,census_division,census_division_recs,building_america_climate_zone,cambium_GEA_region,state,city,county,...,2023_baseline_cooking_consumption,2024_baseline_cooking_consumption,2025_baseline_cooking_consumption,2026_baseline_cooking_consumption,2027_baseline_cooking_consumption,2028_baseline_cooking_consumption,2029_baseline_cooking_consumption,2030_baseline_cooking_consumption,2031_baseline_cooking_consumption,2032_baseline_cooking_consumption
2,239,1690.0,South,East South Central,East South Central,Hot-Humid,SRSOc,AL,Not in a census Place,G0100390,...,979.44,979.44,979.44,979.44,979.44,979.44,979.44,979.44,979.44,979.44
3,273,1690.0,South,East South Central,East South Central,Mixed-Humid,SRSOc,AL,In another census Place,G0100150,...,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20
4,307,1220.0,South,East South Central,East South Central,Hot-Humid,SRSOc,AL,Not in a census Place,G0100850,...,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20
5,409,1220.0,South,East South Central,East South Central,Hot-Humid,SRSOc,AL,Not in a census Place,G0100050,...,1009.63,1009.63,1009.63,1009.63,1009.63,1009.63,1009.63,1009.63,1009.63,1009.63
7,517,1220.0,South,East South Central,East South Central,Mixed-Humid,SRSOc,AL,In another census Place,G0101270,...,377.18,377.18,377.18,377.18,377.18,377.18,377.18,377.18,377.18,377.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548905,548109,1690.0,West,Mountain,Mountain North,Cold,RMPAc,WY,In another census Place,G5600050,...,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61
548907,548226,2176.0,West,Mountain,Mountain North,Cold,RMPAc,WY,In another census Place,G5600050,...,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61
548908,548228,1690.0,West,Mountain,Mountain North,Cold,RMPAc,WY,Not in a census Place,G5600010,...,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61
548910,548417,885.0,West,Mountain,Mountain North,Cold,RMPAc,WY,Casper,G5600250,...,486.50,486.50,486.50,486.50,486.50,486.50,486.50,486.50,486.50,486.50


## Factors to Project Future Energy Consumption Using EIA Heating Degree Day (HDD) Forecasted Data

## Public Perspective: Monetized Marginal Damages from Emissions

### Step 1: Calculate emissions factors for different fuel sources

### Marginal Emissions Factors
#### Electricity
- STATE Regional Aggregation is what is used in the Parth Analysis 
- "Marginal Emissions Factors for Electricity"
- Factor Type: Marginal
- Calculation Method: Regression
- Metric: Emissions [kg/MWh]")
- Predictor: Year")
- Pollutants: SO2, NOx, PM2.5, CO2")
#### Fossil Fuels
- NOx, SO2, CO2: 
    - RESNET Table 7.1.2 Emissions Factors for Household Combustion Fuels
    - Source: https://www.resnet.us/wp-content/uploads/ANSIRESNETICC301-2022_resnetpblshd.pdf
    - All factors are in units of lb/Mbtu so energy consumption in kWh need to be converted to kWh 
    - (1 lb / Mbtu) * (1 Mbtu / 1x10^6 Btu) * (3412 Btu / 1 kWh)
- PM2.5: 
    - A National Methodology and Emission Inventory for Residential Fuel Combustion
    - Source: https://www3.epa.gov/ttnchie1/conference/ei12/area/haneke.pdf

In [8]:
print("""
-------------------------------------------------------------------------------------------------------
Public Perspective: Monetized Marginal Damages from Emissions
-------------------------------------------------------------------------------------------------------
Step 1: Calculate emissions factors for different fuel sources
- Electricity
- Natural Gas
- Fuel Oil 
- Propane
-------------------------------------------------------------------------------------------------------
""")


-------------------------------------------------------------------------------------------------------
Public Perspective: Monetized Marginal Damages from Emissions
-------------------------------------------------------------------------------------------------------
Step 1: Calculate emissions factors for different fuel sources
- Electricity
- Natural Gas
- Fuel Oil 
- Propane
-------------------------------------------------------------------------------------------------------



In [9]:
print("""
-------------------------------------------------------------------------------------------------------
Calculate Emissions Factors: ELECTRICITY
-------------------------------------------------------------------------------------------------------
Electricity Marginal Emissions Factors:
- STATE Regional Aggregation is what is used in the Parth Analysis 
- "Marginal Emissions Factors for Electricity"
- Factor Type: Marginal
- Calculation Method: Regression
- Metric: Emissions [kg/MWh]
- Predictor: Year")
- Pollutants: SO2, NOx, PM2.5, CO2
-------------------------------------------------------------------------------------------------------
""")
filename = 'Generation-MARREG-EMIT-state-byYear.csv'
relative_path = os.path.join(r"margEmis_electricity", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

df_margEmissions = pd.read_csv(file_path, index_col=0)

# Convert from kg/MWh to lb/kWh
# Obtain value from the CSV file and convert to lbs pollutant per kWh 
df_margEmis_electricity = pd.DataFrame({
    'state': df_margEmissions['region'],
    'fuel_type': 'electricity',
    'pollutant': df_margEmissions['pollutant'],
    'value': df_margEmissions['factor'] * (2.20462/1) * (1/1000),
    'unit': '[lb/kWh]'
})
df_margEmis_electricity


-------------------------------------------------------------------------------------------------------
Calculate Emissions Factors: ELECTRICITY
-------------------------------------------------------------------------------------------------------
Electricity Marginal Emissions Factors:
- STATE Regional Aggregation is what is used in the Parth Analysis 
- "Marginal Emissions Factors for Electricity"
- Factor Type: Marginal
- Calculation Method: Regression
- Metric: Emissions [kg/MWh]
- Predictor: Year")
- Pollutants: SO2, NOx, PM2.5, CO2
-------------------------------------------------------------------------------------------------------

Retrieved data for filename: Generation-MARREG-EMIT-state-byYear.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\margEmis_electricity\Generation-MARREG-EMIT-state-byYear.csv




,state,fuel_type,pollutant,value,unit
1,AL,electricity,so2,0.000131,[lb/kWh]
2,AL,electricity,nox,0.000440,[lb/kWh]
3,AL,electricity,pm25,0.000140,[lb/kWh]
4,AL,electricity,co2,1.172667,[lb/kWh]
5,AR,electricity,so2,0.002555,[lb/kWh]
...,...,...,...,...,...
184,WV,electricity,co2,1.713400,[lb/kWh]
185,WY,electricity,so2,0.001223,[lb/kWh]
186,WY,electricity,nox,0.001602,[lb/kWh]
187,WY,electricity,pm25,0.000259,[lb/kWh]


In [10]:
print("""
-------------------------------------------------------------------------------------------------------
Calculate Emissions Factors: FOSSIL FUELS
-------------------------------------------------------------------------------------------------------
Fossil Fuels (Natural Gas, Fuel Oil, Propane):
- NOx, SO2, CO2: 
    - RESNET Table 7.1.2 Emissions Factors for Household Combustion Fuels
    - Source: https://www.resnet.us/wp-content/uploads/ANSIRESNETICC301-2022_resnetpblshd.pdf
    - All factors are in units of lb/Mbtu so energy consumption in kWh need to be converted to kWh 
    - (1 lb / Mbtu) * (1 Mbtu / 1x10^6 Btu) * (3412 Btu / 1 kWh)
- PM2.5: 
    - A National Methodology and Emission Inventory for Residential Fuel Combustion
    - Source: https://www3.epa.gov/ttnchie1/conference/ei12/area/haneke.pdf
-------------------------------------------------------------------------------------------------------
""")

fuelOil_factors = calculate_fossilFuel_emission_factor("fuelOil", 0.0015, 0.1300, 0.83, 161.0, 1000, 138500)
naturalGas_factors = calculate_fossilFuel_emission_factor("naturalGas", 0.0006, 0.0922, 1.9, 117.6, 1000000, 1039)
propane_factors = calculate_fossilFuel_emission_factor("propane", 0.0002, 0.1421, 0.17, 136.6, 1000, 91452)

all_factors = {**fuelOil_factors, **naturalGas_factors, **propane_factors}

df_margEmis_factors = pd.DataFrame.from_dict(all_factors, orient="index", columns=["value"])
df_margEmis_factors.reset_index(inplace=True)
df_margEmis_factors.columns = ["pollutant", "value"]
df_margEmis_factors[["fuel_type", "pollutant"]] = df_margEmis_factors["pollutant"].str.split("_", expand=True)
df_margEmis_factors["unit"] = "[lb/kWh]"

# Add the 'state' column and assign 'National' to every row
df_margEmis_factors = df_margEmis_factors.assign(state='National')

df_margEmis_factors = df_margEmis_factors[["state", "fuel_type", "pollutant", "value", "unit"]]
df_margEmis_factors


-------------------------------------------------------------------------------------------------------
Calculate Emissions Factors: FOSSIL FUELS
-------------------------------------------------------------------------------------------------------
Fossil Fuels (Natural Gas, Fuel Oil, Propane):
- NOx, SO2, CO2: 
    - RESNET Table 7.1.2 Emissions Factors for Household Combustion Fuels
    - Source: https://www.resnet.us/wp-content/uploads/ANSIRESNETICC301-2022_resnetpblshd.pdf
    - All factors are in units of lb/Mbtu so energy consumption in kWh need to be converted to kWh 
    - (1 lb / Mbtu) * (1 Mbtu / 1x10^6 Btu) * (3412 Btu / 1 kWh)
- PM2.5: 
    - A National Methodology and Emission Inventory for Residential Fuel Combustion
    - Source: https://www3.epa.gov/ttnchie1/conference/ei12/area/haneke.pdf
-------------------------------------------------------------------------------------------------------



,state,fuel_type,pollutant,value,unit
0,National,fuelOil,so2,5.118000e-06,[lb/kWh]
1,National,fuelOil,nox,4.435600e-04,[lb/kWh]
2,National,fuelOil,pm25,2.044736e-05,[lb/kWh]
3,National,fuelOil,co2,5.493320e-01,[lb/kWh]
4,National,naturalGas,so2,2.047200e-06,[lb/kWh]
5,National,naturalGas,nox,3.145864e-04,[lb/kWh]
6,National,naturalGas,pm25,6.239461e-06,[lb/kWh]
7,National,naturalGas,co2,4.012512e-01,[lb/kWh]
8,National,propane,so2,6.824000e-07,[lb/kWh]
9,National,propane,nox,4.848452e-04,[lb/kWh]


### Step 2: Adjust Natural Gas & Electricity Emissions Factors for Natural Gas Leakage

In [11]:
print("""
-------------------------------------------------------------------------------------------------------
Step 2: Adjust Natural Gas & Electricity Emissions Factors for Natural Gas Leakage
-------------------------------------------------------------------------------------------------------
Natural Gas (Deetjen et al.): 
"To account for the natural gas infrastructure's leakage of the greenhouse gas methane, 
we estimate the amount of methane leaked per therm of natural gas consumed for heating and 
convert to CO2-equivalent emissions via the GWP of methane. We assume that for every therm of 
natural gas consumed for heating, 0.023 therms of methane escape to the atmosphere [28]. 
Using the energy density of natural gas, we convert from therms to kilograms and multiply 
by 28—the GWP of methane [29]—to calculate a rate of 1.27 kg CO2-equivalent per therm of 
consumed natural gas."

Electricity NERC Regions (Deetjen et al): 
"To account for the natural gas infrastructure's leakage of the greenhouse gas methane, 
we estimate the amount of methane leaked per MWh of electricity generation in each NERC 
region and convert to CO2-equivalent emissions via the global warming potential (GWP) of methane. 
For example, we find that in 2017, the states comprising the western region (WECC) of 
the US electric grid consumed 1.45 million MMcf of natural gas in the power sector [27]. 
We assume that for every MMcf of consumed natural gas, 0.023 MMcf of methane is leaked into 
the atmosphere [28]. By multiplying that leakage rate by the 1.45 million MMcf of consumed 
natural gas, converting to tonnes, and multiplying by a GWP of 28 [29], we estimate 
that the 2017 WECC power sector contributed to methane leakage amounting to 18.6 Mt CO2-equivalent.
By dividing this 18.6 Mt by the 724 TWh of the WECC states' generated electricity [27], we 
calculate a methane leakage rate factor of 25.7 kg MWh−1. In the same manner, we calculate the 
methane leakage rate factors for the other NERC regions. We use the 100 years GWP value of 28 
for methane. Although there have been proposals to use 20 years GWP values, recent research 
shows that the benefits of this alternative 20 years time from are overstated [30]."
-------------------------------------------------------------------------------------------------------
""")
filename = 'natural_gas_leakage_rate.csv'
relative_path = os.path.join(r"margEmis_electricity", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

df_naturalGas_leakage_rate = pd.read_csv(file_path)

state_abbreviations = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'District of Columbia': 'DC',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

# Map full state names to abbreviations
df_naturalGas_leakage_rate['state'] = df_naturalGas_leakage_rate['state_name'].map(state_abbreviations)

# thousand Mcf * (0.023 Mcf leak/1 Mcf) * (19.3 tonnes/1000 Mcf) * (1000 kg/1 tonne) * (2.205 lb/1 kg)) / (thousand MWh * (1000 MWh/thousand MWh)) 
df_naturalGas_leakage_rate['naturalGas_leakage_lbCH4_perMWh'] = (df_naturalGas_leakage_rate['naturalGas_electricity_generation'] * (0.023/1) * (19.3/1) * (1000/1) * (2.205/1)) / (df_naturalGas_leakage_rate['net_generation'] * (1000/1)) 

# (lb CH4/MWh) * (28 lb CO2e/1 lb CH4)
df_naturalGas_leakage_rate['naturalGas_leakage_lbCO2e_perMWh'] = df_naturalGas_leakage_rate['naturalGas_leakage_lbCH4_perMWh'] * (28/1)

# (lb CO2e/MWh) * (1 MWh / 1000 kWh)
df_naturalGas_leakage_rate['naturalGas_leakage_lbCO2e_perkWh'] = df_naturalGas_leakage_rate['naturalGas_leakage_lbCO2e_perMWh'] * (1/1000)
df_naturalGas_leakage_rate


-------------------------------------------------------------------------------------------------------
Step 2: Adjust Natural Gas & Electricity Emissions Factors for Natural Gas Leakage
-------------------------------------------------------------------------------------------------------
Natural Gas (Deetjen et al.): 
"To account for the natural gas infrastructure's leakage of the greenhouse gas methane, 
we estimate the amount of methane leaked per therm of natural gas consumed for heating and 
convert to CO2-equivalent emissions via the GWP of methane. We assume that for every therm of 
natural gas consumed for heating, 0.023 therms of methane escape to the atmosphere [28]. 
Using the energy density of natural gas, we convert from therms to kilograms and multiply 
by 28—the GWP of methane [29]—to calculate a rate of 1.27 kg CO2-equivalent per therm of 
consumed natural gas."

Electricity NERC Regions (Deetjen et al): 
"To account for the natural gas infrastructure's leakage of the

,state_name,naturalGas_electricity_generation,units,net_generation,units.1,state,naturalGas_leakage_lbCH4_perMWh,naturalGas_leakage_lbCO2e_perMWh,naturalGas_leakage_lbCO2e_perkWh
0,Connecticut,135274,thousand Mcf,38376,thousand megawatthours,CT,3.450233,96.606511,0.096607
1,Maine,13718,thousand Mcf,9308,thousand megawatthours,ME,1.442541,40.391148,0.040391
2,Massachusetts,128810,thousand Mcf,26263,thousand megawatthours,MA,4.800638,134.417872,0.134418
3,New Hampshire,21563,thousand Mcf,16988,thousand megawatthours,NH,1.242398,34.787138,0.034787
4,Rhode Island,57260,thousand Mcf,8170,thousand megawatthours,RI,6.859983,192.079518,0.192080
5,Vermont,11,thousand Mcf,2175,thousand megawatthours,VT,0.004950,0.138607,0.000139
6,New Jersey,267122,thousand Mcf,73727,thousand megawatthours,NJ,3.546311,99.296711,0.099297
7,New York,390742,thousand Mcf,130301,thousand megawatthours,NY,2.935189,82.185295,0.082185
8,Pennsylvania,530196,thousand Mcf,212285,thousand megawatthours,PA,2.444617,68.449284,0.068449
9,Illinois,135512,thousand Mcf,184791,thousand megawatthours,IL,0.717779,20.097809,0.020098


In [12]:
# NATURAL GAS LEAKAGE: NATURAL GAS USED IN ELECTRICITY GENERATION
if 'naturalGas_leakage_lbCO2e_perkWh' in df_margEmis_electricity.columns:
    df_margEmis_electricity.drop(columns=['naturalGas_leakage_lbCO2e_perkWh'], inplace=True)

df_margEmis_electricity = df_margEmis_electricity.merge(
    df_naturalGas_leakage_rate[['state', 'naturalGas_leakage_lbCO2e_perkWh']],
    how='left',  # Use a left join to keep all rows from df_margEmis_electricity
    on=['state']  # Merge on the 'state' column
)
# Set 'naturalGas_leakage_lbCO2e_perkWh' to zero where 'pollutant' is not 'co2'
df_margEmis_electricity.loc[df_margEmis_electricity['pollutant'] != 'co2', 'naturalGas_leakage_lbCO2e_perkWh'] = 0.0

# Calculate adjusted marginal emissions factore with natural gas fugitive emissions
df_margEmis_electricity['margEmis_factor_adjusted'] = df_margEmis_electricity['value'] + df_margEmis_electricity['naturalGas_leakage_lbCO2e_perkWh'] 

# Create a factor to multiply marginal damages by
df_margEmis_electricity['naturalGas_leakage_factor'] = df_margEmis_electricity['margEmis_factor_adjusted'] / df_margEmis_electricity['value']

# Reorder columns
df_margEmis_electricity = df_margEmis_electricity[['state', 'fuel_type', 'pollutant', 'value', 'unit', 'naturalGas_leakage_lbCO2e_perkWh', 'margEmis_factor_adjusted', 'naturalGas_leakage_factor']]
df_margEmis_electricity

,state,fuel_type,pollutant,value,unit,naturalGas_leakage_lbCO2e_perkWh,margEmis_factor_adjusted,naturalGas_leakage_factor
0,AL,electricity,so2,0.000131,[lb/kWh],0.000000,0.000131,1.000000
1,AL,electricity,nox,0.000440,[lb/kWh],0.000000,0.000440,1.000000
2,AL,electricity,pm25,0.000140,[lb/kWh],0.000000,0.000140,1.000000
3,AL,electricity,co2,1.172667,[lb/kWh],0.080872,1.253539,1.068964
4,AR,electricity,so2,0.002555,[lb/kWh],0.000000,0.002555,1.000000
...,...,...,...,...,...,...,...,...
183,WV,electricity,co2,1.713400,[lb/kWh],0.004440,1.717840,1.002591
184,WY,electricity,so2,0.001223,[lb/kWh],0.000000,0.001223,1.000000
185,WY,electricity,nox,0.001602,[lb/kWh],0.000000,0.001602,1.000000
186,WY,electricity,pm25,0.000259,[lb/kWh],0.000000,0.000259,1.000000


In [13]:
# NATURAL GAS LEAKAGE: NATURAL GAS INFRASTRUCTURE
# leakage rate for natural gas infrastructure
# 1 Therm = 29.30 kWh --> 1.27 kg CO2e/therm * (1 therm/29.30 kWh) = 0.043 kg CO2e/kWh = 0.095 lb CO2e/kWh
df_margEmis_factors['naturalGas_leakage_lbCO2e_perkWh'] = 0.095

# Set 'naturalGas_leakage_lbCO2e_perkWh' to zero where 'pollutant' is not 'co2'
df_margEmis_factors.loc[df_margEmis_factors['pollutant'] != 'co2', 'naturalGas_leakage_lbCO2e_perkWh'] = 0.0

# Set 'naturalGas_leakage_lbCO2e_perkWh' to zero where 'fuel_type' is not 'naturalGas'
df_margEmis_factors.loc[df_margEmis_factors['fuel_type'] != 'naturalGas', 'naturalGas_leakage_lbCO2e_perkWh'] = 0.0

# Calculate adjusted marginal emissions factor with natural gas fugitive emissions
df_margEmis_factors['margEmis_factor_adjusted'] = df_margEmis_factors['value'] + df_margEmis_factors['naturalGas_leakage_lbCO2e_perkWh'] 

# Create a factor to multiply marginal damages by
df_margEmis_factors['naturalGas_leakage_factor'] = df_margEmis_factors['margEmis_factor_adjusted'] / df_margEmis_factors['value']

# Reorder columns
df_margEmis_factors = df_margEmis_factors[['state', 'fuel_type', 'pollutant', 'value', 'unit', 'naturalGas_leakage_lbCO2e_perkWh', 'margEmis_factor_adjusted', 'naturalGas_leakage_factor']]
df_margEmis_factors

,state,fuel_type,pollutant,value,unit,naturalGas_leakage_lbCO2e_perkWh,margEmis_factor_adjusted,naturalGas_leakage_factor
0,National,fuelOil,so2,5.118000e-06,[lb/kWh],0.000,5.118000e-06,1.000000
1,National,fuelOil,nox,4.435600e-04,[lb/kWh],0.000,4.435600e-04,1.000000
2,National,fuelOil,pm25,2.044736e-05,[lb/kWh],0.000,2.044736e-05,1.000000
3,National,fuelOil,co2,5.493320e-01,[lb/kWh],0.000,5.493320e-01,1.000000
4,National,naturalGas,so2,2.047200e-06,[lb/kWh],0.000,2.047200e-06,1.000000
5,National,naturalGas,nox,3.145864e-04,[lb/kWh],0.000,3.145864e-04,1.000000
6,National,naturalGas,pm25,6.239461e-06,[lb/kWh],0.000,6.239461e-06,1.000000
7,National,naturalGas,co2,4.012512e-01,[lb/kWh],0.095,4.962512e-01,1.236759
8,National,propane,so2,6.824000e-07,[lb/kWh],0.000,6.824000e-07,1.000000
9,National,propane,nox,4.848452e-04,[lb/kWh],0.000,4.848452e-04,1.000000


In [14]:
# Append df_margEmissions_electricity to df_margEmis_factors
# This produces a dataframe of marginal emissions rates for various fuel types
df_margEmis_factors = pd.concat([df_margEmis_factors, df_margEmis_electricity], ignore_index=True)
df_margEmis_factors

,state,fuel_type,pollutant,value,unit,naturalGas_leakage_lbCO2e_perkWh,margEmis_factor_adjusted,naturalGas_leakage_factor
0,National,fuelOil,so2,0.000005,[lb/kWh],0.000000,0.000005,1.000000
1,National,fuelOil,nox,0.000444,[lb/kWh],0.000000,0.000444,1.000000
2,National,fuelOil,pm25,0.000020,[lb/kWh],0.000000,0.000020,1.000000
3,National,fuelOil,co2,0.549332,[lb/kWh],0.000000,0.549332,1.000000
4,National,naturalGas,so2,0.000002,[lb/kWh],0.000000,0.000002,1.000000
...,...,...,...,...,...,...,...,...
195,WV,electricity,co2,1.713400,[lb/kWh],0.004440,1.717840,1.002591
196,WY,electricity,so2,0.001223,[lb/kWh],0.000000,0.001223,1.000000
197,WY,electricity,nox,0.001602,[lb/kWh],0.000000,0.001602,1.000000
198,WY,electricity,pm25,0.000259,[lb/kWh],0.000000,0.000259,1.000000


### Step 3: Quantify monitized damages using EASIUR Marginal Social Cost Factors
#### THE STEPS BELOW SUMMARIZE WHAT WAS DONE TO OBTAIN ALL NATIONAL EASIUR VALUES INCLUDED ON GITHUB
- Obtain all of the dwelling unit latitude and longitude values from the metadata columns
- Make a new dataframe of just the longitude and latitude values 
    - Make sure that the order is (longitude, latitude)
    - Do not include the index or column name when exporting 
- Export the CSV
- **Upload csv to EASIUR Website:**
    - Website: https://barney.ce.cmu.edu/~jinhyok/easiur/online/
    - See inputs in respective sections
- Download the file and put it in the 'easiur_batchConversion_download' folder
- Copy and paste the name of the file EASIUR generated when prompted
- Copy and paste the name of the filepath for the 'easiur_batchConversion_download' folder when prompted
- Match up the longitude and latitudes for each dwelling unit with the selected damages

### Fossil Fuels: EASIUR Marginal Damage (Social Cost) Factors Info
- Factor Type: Marginal Social Cost
- Calculation Method: Regression
- Metric: Marginal Social Cost [USD per metric ton]
- Dollar Year: 2010
- Income Year: 2018
- Population Year: 2018
- Aggregation: Longitude, and Latitude Coordinates
- Pollutants: Primary PM2.5, Sulfur Dioxide (SO2), Nitrogen Oxides (NOx), Ammonia (NH3)
- Elevation (Ground, 150m, 300m) and Seasons (Winter, Spring, Summer, Fall)

In [15]:
# Create a dataframe containing just the longitude and Latitude
df_EASIUR_batchConversion = pd.DataFrame({
    'Longitude':df_euss_am_baseline['in.weather_file_longitude'],
    'Latitude':df_euss_am_baseline['in.weather_file_latitude'],
})

# Drop duplicate rows based on 'Longitude' and 'Latitude' columns
df_EASIUR_batchConversion.drop_duplicates(subset=['Longitude', 'Latitude'], keep='first', inplace=True)

# Create a location ID for the name of the batch conversion file
while True:
    if menu_state == 'N':
        location_id = 'National'
        print("You chose to analyze all of the United States.")
        break
    elif menu_state == 'Y':
        if menu_city == 'N':
            try:
                location_id = str(input_state)
                print(f"Location ID is: {location_id}")
                break
            except ValueError:
                print("Invalid input for state!")
        elif menu_city == 'Y':
            try:
                location_id = input_cityFilter.replace(', ', '_').strip()
                print(f"Location ID is: {location_id}")
                break
            except AttributeError:
                print("Invalid input for city filter!")
        else:
            print("Incorrect state or city filter assignment!")
    else:
        print("Invalid data location. Check your inputs at the beginning of this notebook!")

# Updated GitHub code has EASIUR file with all unique latitude, longitude coordinates in the US
filename = 'easiur_National2024-06-1420-17.csv'
relative_path = os.path.join(r"margDamages_EASIUR\easiur_batchConversion_download", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

df_margSocialCosts = pd.read_csv(file_path)

# Convert from kg/MWh to lb/kWh
# Obtain value from the CSV file and convert to lbs pollutant per kWh 
# Inflate from 2010 to 2018

# Define df_margSocialCosts_EASIUR DataFrame first
df_margSocialCosts_EASIUR = pd.DataFrame({
    'Longitude': df_margSocialCosts['Longitude'],
    'Latitude': df_margSocialCosts['Latitude']
})
df_margSocialCosts_EASIUR

You chose to analyze all of the United States.
Retrieved data for filename: easiur_National2024-06-1420-17.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\margDamages_EASIUR\easiur_batchConversion_download\easiur_National2024-06-1420-17.csv




,Longitude,Latitude
0,-87.04,31.42
1,-85.86,33.59
2,-86.39,32.30
3,-85.45,31.32
4,-86.75,33.56
...,...,...
1024,-106.72,44.38
1025,-105.54,44.34
1026,-107.95,43.97
1027,-108.08,44.52


### Step 4: Inflate Marginal Social Cost (Damage) Factors using BLS CPI for All Urban Consumers (CPI-U)
- Series Id:	CUUR0000SA0
- Not Seasonally Adjusted
- Series Title:	All items in U.S. city average, all urban consumers, not seasonally adjusted
- Area:	U.S. city average
- Item:	All items
- Base Period:	1982-84=100

### Use the updated Social Cost of Carbon (190 USD-2020/ton CO2) and inflate to USD-2021
- EPA Median for 2% near term discount rate and most commonly mentioned value is 190 USD-2020 using the GIVE model.
- 190 USD-2020 has some inconsistency with the VSL being used. An old study and 2008 VSL is noted
- 190 USD value and inflate to USD 2021 because there is a clear source and ease of replicability.

### Adjustment for VSL
- EASIUR uses a VSL of 8.8M USD-2010 
- New EPA VSL is 11.3M USD-2021

In [16]:
# Load the BLS Inflation Data
filename = 'bls_cpiu_2005-2023.xlsx'
relative_path = os.path.join(r"inflation_data", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

# Create a pandas dataframe
df_bls_cpiu = pd.read_excel(file_path, sheet_name='bls_cpiu')

df_bls_cpiu = pd.DataFrame({
    'year': df_bls_cpiu['Year'],
    'cpiu_annual': df_bls_cpiu['Annual']
})

# Obtain the Annual CPIU values for the years of interest
bls_cpi_annual_2008 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2008)].item()
bls_cpi_annual_2010 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2010)].item()
bls_cpi_annual_2013 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2013)].item()
bls_cpi_annual_2018 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2018)].item()
bls_cpi_annual_2019 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2019)].item()
bls_cpi_annual_2020 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2020)].item()
bls_cpi_annual_2021 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2021)].item()
bls_cpi_annual_2022 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2022)].item()

# Precompute constant values
cpi_ratio_2021_2022 = bls_cpi_annual_2021 / bls_cpi_annual_2022
cpi_ratio_2021_2021 = bls_cpi_annual_2021 / bls_cpi_annual_2021  # This will be 1
cpi_ratio_2021_2020 = bls_cpi_annual_2021 / bls_cpi_annual_2020  # For SCC
cpi_ratio_2021_2019 = bls_cpi_annual_2021 / bls_cpi_annual_2019 
cpi_ratio_2021_2018 = bls_cpi_annual_2021 / bls_cpi_annual_2018 
cpi_ratio_2021_2013 = bls_cpi_annual_2021 / bls_cpi_annual_2013
cpi_ratio_2021_2010 = bls_cpi_annual_2021 / bls_cpi_annual_2010
cpi_ratio_2021_2008 = bls_cpi_annual_2021 / bls_cpi_annual_2008  # For EPA VSL and SCC

# 2021 US EPA VSL is $11.3M in 2021 USD
df_margSocialCosts_EASIUR['current_VSL_USD2021'] = 11.3

# Easiur uses a VSL of $8.8 M USD2010
# Inflate to 2021 $USD
df_margSocialCosts_EASIUR['easiur_VSL_USD2021'] = 8.8 * cpi_ratio_2021_2010

# Use df_margSocialCosts_EASIUR in the calculation of other columns
# Also adjust the VSL
df_margSocialCosts_EASIUR['margSocialCosts_pm25'] = round((df_margSocialCosts['PM25 Annual Ground'] * (1/2204.6) * (df_margSocialCosts_EASIUR['current_VSL_USD2021']/df_margSocialCosts_EASIUR['easiur_VSL_USD2021'])), 2)
df_margSocialCosts_EASIUR['margSocialCosts_so2'] = round((df_margSocialCosts['SO2 Annual Ground'] * (1/2204.6) * (df_margSocialCosts_EASIUR['current_VSL_USD2021']/df_margSocialCosts_EASIUR['easiur_VSL_USD2021'])), 2)
df_margSocialCosts_EASIUR['margSocialCosts_nox'] = round((df_margSocialCosts['NOX Annual Ground'] * (1/2204.6) * (df_margSocialCosts_EASIUR['current_VSL_USD2021']/df_margSocialCosts_EASIUR['easiur_VSL_USD2021'])), 2)

# Note that SCC of $190 USD-2020 has some inconsistency with the VSL being used. An old study and 2008 VSL is noted
# We use the $190 USD value and inflate to USD 2021 because there is a clear source and ease of replicability.
df_margSocialCosts_EASIUR['margSocialCosts_co2'] = round((190 * cpi_ratio_2021_2020 * (1/2204.6)), 2)
df_margSocialCosts_EASIUR['unit'] = '[$USD2021/lb]'
df_margSocialCosts_EASIUR

Retrieved data for filename: bls_cpiu_2005-2023.xlsx
Located at filepath: c:\Users\14128\Research\cmu-tare-model\inflation_data\bls_cpiu_2005-2023.xlsx


,Longitude,Latitude,current_VSL_USD2021,easiur_VSL_USD2021,margSocialCosts_pm25,margSocialCosts_so2,margSocialCosts_nox,margSocialCosts_co2,unit
0,-87.04,31.42,11.3,10.935429,36.37,8.72,1.63,0.09,[$USD2021/lb]
1,-85.86,33.59,11.3,10.935429,54.55,9.56,2.53,0.09,[$USD2021/lb]
2,-86.39,32.30,11.3,10.935429,49.71,9.08,2.05,0.09,[$USD2021/lb]
3,-85.45,31.32,11.3,10.935429,41.54,9.09,1.79,0.09,[$USD2021/lb]
4,-86.75,33.56,11.3,10.935429,94.13,10.51,2.89,0.09,[$USD2021/lb]
...,...,...,...,...,...,...,...,...,...
1024,-106.72,44.38,11.3,10.935429,9.87,4.69,0.74,0.09,[$USD2021/lb]
1025,-105.54,44.34,11.3,10.935429,10.93,5.64,0.93,0.09,[$USD2021/lb]
1026,-107.95,43.97,11.3,10.935429,8.51,4.20,0.59,0.09,[$USD2021/lb]
1027,-108.08,44.52,11.3,10.935429,8.90,5.37,0.79,0.09,[$USD2021/lb]


## Electricity CEDM-EASIUR Marginal Damages: Current and Decarbonizing Grid
- Factor Type: Marginal
- Calculation Method: Regression
- Metric: Marginal Damages EASIUR [USD per MWh or kWh]
- Year: 2018
- Regional Aggregation: eGRID subregion (all regions)
- Pollutants: SO2, NOx, PM2.5 CO2

SCC Adjustment: We use the EPA suggested 190 USD-2020 value for the social cost of carbon and inflate to 2021 USD. 

VSL: "We use a value of a statistical life (VSL) of USD 8.8 million (in 2010 dollars) for both our AP2 and EASIUR calculations. EASIUR reports damage intensities in USD/metric ton using this VSL and dollar year."

In [17]:
# For CO2 adjust SCC
# Create an adjustment factor for the new Social Cost of Carbon (SCC)
epa_scc = 190 * cpi_ratio_2021_2020
old_scc = 40 * cpi_ratio_2021_2010
scc_adjustment_factor = epa_scc / old_scc

# For Health-Related Emissions Adjust for different Value of a Statistical Life (VSL) values
# Current VSL is $11.3 M USD2021
current_VSL_USD2021 = 11.3

# Easiur uses a VSL of $8.8 M USD2010
easiur_VSL_USD2021 = 8.8 * (cpi_ratio_2021_2010)

# Calculate VSL adjustment factor
vsl_adjustment_factor = current_VSL_USD2021 / easiur_VSL_USD2021

### Damages from Climate Related Emissions

In [18]:
# Climate damages (co2) are expected to decline 68% linearlly by 2030 (% relative to 2005)
# Note only 2006 data available, used in place of 2005
filename = 'Generation-MARREG-DAMEASIUR-egrid-byYear_climate2006.csv'
relative_path = os.path.join(r"margDamages_EASIUR", filename)
file_path = os.path.join(project_root, relative_path)
df_margDamages_climate2006 = pd.read_csv(file_path, index_col=0)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

Retrieved data for filename: Generation-MARREG-DAMEASIUR-egrid-byYear_climate2006.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\margDamages_EASIUR\Generation-MARREG-DAMEASIUR-egrid-byYear_climate2006.csv


In [19]:
# Climate damages (co2) are expected to decline 68% linearlly by 2030 (% relative to 2005)
# Note 2018 start year
filename = 'Generation-MARREG-DAMEASIUR-egrid-byYear_climate2018.csv'
relative_path = os.path.join(r"margDamages_EASIUR", filename)
file_path = os.path.join(project_root, relative_path)
df_margDamages_climate2018 = pd.read_csv(file_path, index_col=0)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

Retrieved data for filename: Generation-MARREG-DAMEASIUR-egrid-byYear_climate2018.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\margDamages_EASIUR\Generation-MARREG-DAMEASIUR-egrid-byYear_climate2018.csv


In [20]:
# Marginal damages [$/kWh]
# Inflate from 2010 to 2021
# Note only 2006 data available, used in place of 2005
df_margDamages_EASIUR_climate = pd.DataFrame({
    'subregion_eGRID': df_margDamages_climate2006['region'],
    'pollutant': df_margDamages_climate2006['pollutant'],
    'unit': '[$/kWh]',
    '2030_decarb': '68% from 2005',
    'margDamages_dollarPerkWh_adjustVSL_ref': (df_margDamages_climate2006['factor'] * (scc_adjustment_factor) * (1/1000)) * (cpi_ratio_2021_2010),
    'margDamages_dollarPerkWh_adjustVSL_2018': (df_margDamages_climate2018['factor'] * (scc_adjustment_factor) * (1/1000)) * (cpi_ratio_2021_2010)
})
df_margDamages_EASIUR_climate['margDamages_decarb_2030'] = df_margDamages_EASIUR_climate['margDamages_dollarPerkWh_adjustVSL_ref'] - (df_margDamages_EASIUR_climate['margDamages_dollarPerkWh_adjustVSL_ref'] * 0.68)
df_margDamages_EASIUR_climate['reduction_margDamages_2030'] = df_margDamages_EASIUR_climate['margDamages_dollarPerkWh_adjustVSL_2018'] - df_margDamages_EASIUR_climate['margDamages_decarb_2030']
df_margDamages_EASIUR_climate['reduction_margDamages_annual'] = df_margDamages_EASIUR_climate['reduction_margDamages_2030'] / 12 # Relative to 2018, 
# df_margDamages_EASIUR_climate

### Damages from Health Related Emissions

In [21]:
# Health damages (SO2, NOx, PM2.5) are expected to decline 65% by 2030 (% relative from 2021)
filename = 'Generation-MARREG-DAMEASIUR-egrid-byYear_health2018.csv'
relative_path = os.path.join(r"margDamages_EASIUR", filename)
file_path = os.path.join(project_root, relative_path)
df_margDamages_health2018 = pd.read_csv(file_path, index_col=0)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

Retrieved data for filename: Generation-MARREG-DAMEASIUR-egrid-byYear_health2018.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\margDamages_EASIUR\Generation-MARREG-DAMEASIUR-egrid-byYear_health2018.csv


In [22]:
# Marginal damages [$/kWh]
# Inflate from 2010 to 2021
# Note only 2018 data available, used in place of 2021
df_margDamages_EASIUR_health = pd.DataFrame({
    'subregion_eGRID': df_margDamages_health2018['region'],
    'pollutant': df_margDamages_health2018['pollutant'],
    'unit': '[$/kWh]',
    '2030_decarb': '65% from 2021',
    'margDamages_dollarPerkWh_adjustVSL_ref': (df_margDamages_health2018['factor'] * (vsl_adjustment_factor) * (1/1000)) * (cpi_ratio_2021_2010),
    'margDamages_dollarPerkWh_adjustVSL_2018': (df_margDamages_health2018['factor'] * (vsl_adjustment_factor) * (1/1000)) * (cpi_ratio_2021_2010)
})
df_margDamages_EASIUR_health['margDamages_decarb_2030'] = df_margDamages_EASIUR_health['margDamages_dollarPerkWh_adjustVSL_ref'] - (df_margDamages_EASIUR_health['margDamages_dollarPerkWh_adjustVSL_ref'] * 0.65)
df_margDamages_EASIUR_health['reduction_margDamages_2030'] = df_margDamages_EASIUR_health['margDamages_dollarPerkWh_adjustVSL_2018'] - df_margDamages_EASIUR_health['margDamages_decarb_2030']
df_margDamages_EASIUR_health['reduction_margDamages_annual'] = df_margDamages_EASIUR_health['reduction_margDamages_2030'] / 9
# df_margDamages_EASIUR_health

In [23]:
# Combine them top to bottom
df_margDamages_EASIUR = pd.concat([df_margDamages_EASIUR_climate, df_margDamages_EASIUR_health], ignore_index=True)
df_margDamages_EASIUR

,subregion_eGRID,pollutant,unit,2030_decarb,margDamages_dollarPerkWh_adjustVSL_ref,margDamages_dollarPerkWh_adjustVSL_2018,margDamages_decarb_2030,reduction_margDamages_2030,reduction_margDamages_annual
0,AZNM,co2,[$/kWh],68% from 2005,0.093359,0.107904,0.029875,0.078029,0.006502
1,CAMX,co2,[$/kWh],68% from 2005,0.088012,0.088766,0.028164,0.060603,0.005050
2,ERCT,co2,[$/kWh],68% from 2005,0.106521,0.110424,0.034087,0.076338,0.006361
3,FRCC,co2,[$/kWh],68% from 2005,0.122589,0.093406,0.039228,0.054177,0.004515
4,MROE,co2,[$/kWh],68% from 2005,0.158571,0.158156,0.050743,0.107413,0.008951
...,...,...,...,...,...,...,...,...,...
83,SRTV,nox,[$/kWh],65% from 2021,0.002472,0.002472,0.000865,0.001607,0.000179
84,SRTV,pm25,[$/kWh],65% from 2021,0.006606,0.006606,0.002312,0.004294,0.000477
85,SRVC,so2,[$/kWh],65% from 2021,0.007316,0.007316,0.002560,0.004755,0.000528
86,SRVC,nox,[$/kWh],65% from 2021,0.002162,0.002162,0.000757,0.001405,0.000156


In [24]:
# Marginal Damages for a Gradually Decarbonizing Grid
df_margDamages_gridDecarb = df_margDamages_EASIUR.copy()

years = list(range(2019, 2051))

# Apply reductions
for year in years:
    column_name = f'margDamages_dollarPerkWh_adjustVSL_{year}'
    df_margDamages_gridDecarb[column_name] = df_margDamages_gridDecarb['margDamages_dollarPerkWh_adjustVSL_ref']  # Initialize

    for index, row in df_margDamages_gridDecarb.iterrows():  # Correctly unpack the index and row
        if year <= 2030:
            # Climate reduction (C02) applicable from 2019 to 2030
            # No Health reductions before 2022
            if 2019 <= year < 2022:
                if row['pollutant'] == 'co2':
                    df_margDamages_gridDecarb.at[index, column_name] = df_margDamages_gridDecarb.at[index, f'margDamages_dollarPerkWh_adjustVSL_{year-1}'] - df_margDamages_gridDecarb.at[index, 'reduction_margDamages_annual']
                else:
                    df_margDamages_gridDecarb.at[index, column_name] = df_margDamages_gridDecarb.at[index, f'margDamages_dollarPerkWh_adjustVSL_{year-1}']
            
            # Health reduction applicable from 2022 to 2030
            # Climate reductions continue
            elif year >= 2022:
                df_margDamages_gridDecarb.at[index, column_name] = df_margDamages_gridDecarb.at[index, f'margDamages_dollarPerkWh_adjustVSL_{year-1}'] - df_margDamages_gridDecarb.at[index, 'reduction_margDamages_annual']

        # Post-2030, damage values should be at the 2030 level
        else:
            df_margDamages_gridDecarb.at[index, column_name] = df_margDamages_gridDecarb.at[index, f'margDamages_dollarPerkWh_adjustVSL_2030']

df_margDamages_gridDecarb

,subregion_eGRID,pollutant,unit,2030_decarb,margDamages_dollarPerkWh_adjustVSL_ref,margDamages_dollarPerkWh_adjustVSL_2018,margDamages_decarb_2030,reduction_margDamages_2030,reduction_margDamages_annual,margDamages_dollarPerkWh_adjustVSL_2019,...,margDamages_dollarPerkWh_adjustVSL_2041,margDamages_dollarPerkWh_adjustVSL_2042,margDamages_dollarPerkWh_adjustVSL_2043,margDamages_dollarPerkWh_adjustVSL_2044,margDamages_dollarPerkWh_adjustVSL_2045,margDamages_dollarPerkWh_adjustVSL_2046,margDamages_dollarPerkWh_adjustVSL_2047,margDamages_dollarPerkWh_adjustVSL_2048,margDamages_dollarPerkWh_adjustVSL_2049,margDamages_dollarPerkWh_adjustVSL_2050
0,AZNM,co2,[$/kWh],68% from 2005,0.093359,0.107904,0.029875,0.078029,0.006502,0.101401,...,0.029875,0.029875,0.029875,0.029875,0.029875,0.029875,0.029875,0.029875,0.029875,0.029875
1,CAMX,co2,[$/kWh],68% from 2005,0.088012,0.088766,0.028164,0.060603,0.005050,0.083716,...,0.028164,0.028164,0.028164,0.028164,0.028164,0.028164,0.028164,0.028164,0.028164,0.028164
2,ERCT,co2,[$/kWh],68% from 2005,0.106521,0.110424,0.034087,0.076338,0.006361,0.104063,...,0.034087,0.034087,0.034087,0.034087,0.034087,0.034087,0.034087,0.034087,0.034087,0.034087
3,FRCC,co2,[$/kWh],68% from 2005,0.122589,0.093406,0.039228,0.054177,0.004515,0.088891,...,0.039228,0.039228,0.039228,0.039228,0.039228,0.039228,0.039228,0.039228,0.039228,0.039228
4,MROE,co2,[$/kWh],68% from 2005,0.158571,0.158156,0.050743,0.107413,0.008951,0.149204,...,0.050743,0.050743,0.050743,0.050743,0.050743,0.050743,0.050743,0.050743,0.050743,0.050743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,SRTV,nox,[$/kWh],65% from 2021,0.002472,0.002472,0.000865,0.001607,0.000179,0.002472,...,0.000865,0.000865,0.000865,0.000865,0.000865,0.000865,0.000865,0.000865,0.000865,0.000865
84,SRTV,pm25,[$/kWh],65% from 2021,0.006606,0.006606,0.002312,0.004294,0.000477,0.006606,...,0.002312,0.002312,0.002312,0.002312,0.002312,0.002312,0.002312,0.002312,0.002312,0.002312
85,SRVC,so2,[$/kWh],65% from 2021,0.007316,0.007316,0.002560,0.004755,0.000528,0.007316,...,0.002560,0.002560,0.002560,0.002560,0.002560,0.002560,0.002560,0.002560,0.002560,0.002560
86,SRVC,nox,[$/kWh],65% from 2021,0.002162,0.002162,0.000757,0.001405,0.000156,0.002162,...,0.000757,0.000757,0.000757,0.000757,0.000757,0.000757,0.000757,0.000757,0.000757,0.000757


In [25]:
# Create an empty dictionary to store the lookup data
dict_margDamages_gridDecarb = {}

for year in years:
    # Create an empty dictionary for the current year
    year_lookup = {}
    
    for _, row in df_margDamages_gridDecarb.iterrows():
        year_lookup[(row['subregion_eGRID'], row['pollutant'])] = row[f'margDamages_dollarPerkWh_adjustVSL_{str(year)}']
    
    # Add the year-specific lookup to the main lookup_data dictionary
    dict_margDamages_gridDecarb[year] = year_lookup

# Now, you have a lookup_data dictionary containing emissions factors for each state and year
dict_margDamages_gridDecarb

{2019: {('AZNM', 'co2'): 0.10140113554892047,
  ('CAMX', 'co2'): 0.08371628166873829,
  ('ERCT', 'co2'): 0.10406285163065675,
  ('FRCC', 'co2'): 0.08889105567336082,
  ('MROE', 'co2'): 0.14920444426441548,
  ('MROW', 'co2'): 0.14842635622351424,
  ('NEWE', 'co2'): 0.08180333245678206,
  ('NWPP', 'co2'): 0.129662604747659,
  ('NYCW', 'co2'): 0.08055916500106658,
  ('NYLI', 'co2'): 0.10338846929994001,
  ('NYUP', 'co2'): 0.08115292164008463,
  ('RFCE', 'co2'): 0.09933715524797929,
  ('RFCM', 'co2'): 0.12373499231743167,
  ('RFCW', 'co2'): 0.12982803965521555,
  ('RMPA', 'co2'): 0.1231835790161593,
  ('SPNO', 'co2'): 0.13645288226626034,
  ('SPSO', 'co2'): 0.11873459606535809,
  ('SRMV', 'co2'): 0.10540715212899736,
  ('SRMW', 'co2'): 0.14122130511570125,
  ('SRSO', 'co2'): 0.11514028525910261,
  ('SRTV', 'co2'): 0.11815691807773179,
  ('SRVC', 'co2'): 0.1196891414144477,
  ('AZNM', 'so2'): 0.001565519163910798,
  ('AZNM', 'nox'): 0.000511358363683762,
  ('AZNM', 'pm25'): 0.00180169834374

In [26]:
print("""
-------------------------------------------------------------------------------------------------------
Calculate Emissions Factors: FOSSIL FUELS
-------------------------------------------------------------------------------------------------------
""")

# Create a lookup dictionary for the national emissions factors
national_factors = df_margEmis_factors[df_margEmis_factors['state'] == 'National']
national_lookup = {(row['fuel_type'], row['pollutant']): row['margEmis_factor_adjusted'] for _, row in national_factors.iterrows()}

# Create a lookup dictionary for the state-specific emissions factors for electricity
electricity_factors = df_margEmis_factors[df_margEmis_factors['fuel_type'] == 'electricity']
electricity_lookup = {(row['pollutant'], row['state']): row['margEmis_factor_adjusted'] for _, row in electricity_factors.iterrows()}

pollutants = ['so2', 'nox', 'pm25', 'co2']

# ELECTRICITY CEDM DAMAGES LOOKUP
damages_CEDM_lookup = {(row['pollutant'], row['subregion_eGRID']): row['margDamages_dollarPerkWh_adjustVSL_ref'] for _, row in df_margDamages_EASIUR.iterrows()}

# FOSSIL FUELS DAMAGES LOOKUP
# Create a damages_fossilFuel_lookup dictionary from df_margSocialCosts_EASIUR
damages_fossilFuel_lookup = df_margSocialCosts_EASIUR.groupby(['Longitude', 'Latitude']).first().to_dict()


-------------------------------------------------------------------------------------------------------
Calculate Emissions Factors: FOSSIL FUELS
-------------------------------------------------------------------------------------------------------



### Step 5: Calculate End-use specific marginal damages
**I used the total emissions column for each of the end uses for the following reasons:**
- Most homes only have 1 of each end-use, so it is unlikely that the homes have a significant consumption values from different fuel types. Thus, the total consumption and total emissions column (sum of each dwelling units consumption by end-use for each fuel) is fine to use to calculate marginal damages (social cost)
- We can visualize the emissions in 2 by 2 grid (CO2, PM25, SO2, NOx) with each appliance's heating fuel in a different shape or color. 

### Baseline Marginal Damages: WHOLE-HOME

In [27]:
print("""
-------------------------------------------------------------------------------------------------------
Step 5: Calculate End-use specific marginal damages
-------------------------------------------------------------------------------------------------------
      
-------------------------------------------------------------------------------------------------------
Baseline Marginal Damages: WHOLE-HOME
-------------------------------------------------------------------------------------------------------
""")

# calculate_marginal_damages(df, grid_decarb=False)
df_euss_am_baseline_home = calculate_marginal_damages(df=df_euss_am_baseline_home,
                                                      grid_decarb=False
                                                     )
df_euss_am_baseline_home


-------------------------------------------------------------------------------------------------------
Step 5: Calculate End-use specific marginal damages
-------------------------------------------------------------------------------------------------------
      
-------------------------------------------------------------------------------------------------------
Baseline Marginal Damages: WHOLE-HOME
-------------------------------------------------------------------------------------------------------

End-use category: heating
Baseline Damages for heating - so2:
2    6.12
3    9.93
4    4.24
5    0.03
7    7.47
Name: baseline_heating_damages_so2, dtype: float64
Baseline Damages for heating - nox:
2    3.83
3    8.80
4    3.21
5    0.77
7    6.88
Name: baseline_heating_damages_nox, dtype: float64
Baseline Damages for heating - pm25:
2    27.25
3    60.54
4    24.83
5     0.36
7    71.44
Name: baseline_heating_damages_pm25, dtype: float64
Baseline Damages for heating - co2:
2    

,bldg_id,square_footage,census_region,census_division,census_division_recs,building_america_climate_zone,cambium_GEA_region,state,city,county,...,baseline_clothesDrying_damages_pm25,baseline_clothesDrying_damages_co2,baseline_clothesDrying_damages_health,baseline_clothesDrying_damages_climate,baseline_cooking_damages_so2,baseline_cooking_damages_nox,baseline_cooking_damages_pm25,baseline_cooking_damages_co2,baseline_cooking_damages_health,baseline_cooking_damages_climate
2,239,1690.0,South,East South Central,East South Central,Hot-Humid,SRSOc,AL,Not in a census Place,G0100390,...,2.85,62.96,3.89,62.96,0.06,0.51,0.39,46.17,0.96,46.17
3,273,1690.0,South,East South Central,East South Central,Mixed-Humid,SRSOc,AL,In another census Place,G0100150,...,10.08,148.47,13.20,148.47,0.72,0.64,4.38,64.47,5.74,64.47
4,307,1220.0,South,East South Central,East South Central,Hot-Humid,SRSOc,AL,Not in a census Place,G0100850,...,4.80,77.59,6.24,77.59,0.68,0.52,3.99,64.47,5.19,64.47
5,409,1220.0,South,East South Central,East South Central,Hot-Humid,SRSOc,AL,Not in a census Place,G0100050,...,2.98,57.62,3.99,57.62,0.06,0.58,0.46,47.61,1.10,47.61
7,517,1220.0,South,East South Central,East South Central,Mixed-Humid,SRSOc,AL,In another census Place,G0101270,...,5.90,50.37,7.09,50.37,0.55,0.51,5.31,45.27,6.37,45.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548905,548109,1690.0,West,Mountain,Mountain North,Cold,RMPAc,WY,In another census Place,G5600050,...,2.28,150.32,9.04,150.32,2.98,0.64,1.22,80.42,4.84,80.42
548907,548226,2176.0,West,Mountain,Mountain North,Cold,RMPAc,WY,In another census Place,G5600050,...,1.20,79.31,4.77,79.31,2.98,0.64,1.22,80.42,4.84,80.42
548908,548228,1690.0,West,Mountain,Mountain North,Cold,RMPAc,WY,Not in a census Place,G5600010,...,1.44,79.31,4.16,79.31,2.31,0.44,1.46,80.42,4.21,80.42
548910,548417,885.0,West,Mountain,Mountain North,Cold,RMPAc,WY,Casper,G5600250,...,3.67,181.46,10.37,181.46,3.01,0.55,1.95,96.46,5.51,96.46


## Private Perspective: Annual Energy Costs

### Step 1: Obtain Level Energy Fuel Cost Data from the EIA
**Data Sources for Excel workbook containing state average Residential fuel cost for each fuel in 2018**
- EIA State Electricity Price: https://www.eia.gov/electricity/state/archive/2018/
- EIA Natural Gas Prices: https://www.eia.gov/dnav/ng/ng_pri_sum_dcu_SPA_a.htm
- Propane and Fuel Oil: EIA March 2023 Short Term Energy Outlook
    - https://www.eia.gov/outlooks/steo/pdf/wf01.pdf
    - Table WF01: Average Consumer Prices and Expenditures for Heating Fuels During the Winter
    - US Average: 2018-2019 Data

In [28]:
print("""
-------------------------------------------------------------------------------------------------------
Private Perspective: Annual Energy Costs
-------------------------------------------------------------------------------------------------------
- Step 1: Obtain Level Energy Fuel Cost Data from the EIA
- Step 2: Calculate Annual Operating (Fuel) Costs
-------------------------------------------------------------------------------------------------------
      
-------------------------------------------------------------------------------------------------------
Step 1: Obtain Level Energy Fuel Cost Data from the EIA
-------------------------------------------------------------------------------------------------------
**Data Sources for Excel workbook containing state average Residential fuel cost for each fuel in 2018**
- EIA State Electricity Price: https://www.eia.gov/electricity/state/archive/2018/
- EIA Natural Gas Prices: https://www.eia.gov/dnav/ng/ng_pri_sum_dcu_SPA_a.htm
- Propane and Fuel Oil: EIA March 2023 Short Term Energy Outlook
    - https://www.eia.gov/outlooks/steo/pdf/wf01.pdf
    - Table WF01: Average Consumer Prices and Expenditures for Heating Fuels During the Winter
    - US Average: 2018-2019 Data
-------------------------------------------------------------------------------------------------------
""")

filename = 'fuel_prices_nominal.csv'
relative_path = os.path.join(r"fuel_prices", filename)
file_path = os.path.join(project_root, relative_path)
df_fuelPrices_perkWh = pd.read_csv(file_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

# New units for the converted and inflated prices below
df_fuelPrices_perkWh['units'] = 'USD2021 per kWh'

years = ['2018', '2019', '2020', '2021', '2022']

# Take dataframe with nominal prices in their base units and convert to $/kWh equivalent
# https://www.eia.gov/energyexplained/units-and-calculators/british-thermal-units.php
for year in years:
    for index, row in df_fuelPrices_perkWh.iterrows():
        
        # Propane: (dollars per gallon) * (1 gallon propane/91,452 BTU) * (3412 BTU/1 kWh)
        if row['fuel_type'] == 'propane':
            df_fuelPrices_perkWh.at[index, f'{year}_fuelPrice_perkWh'] = row[f'{year}_nominal_unit_price'] * (1/91452) * (3412/1)
        
        # Fuel Oil: (dollars/gallon) * (1 gallon heating oil/138,500 BTU) * (3412 BTU/1 kWh)
        elif row['fuel_type'] == 'fuelOil':
            df_fuelPrices_perkWh.at[index, f'{year}_fuelPrice_perkWh'] = row[f'{year}_nominal_unit_price'] * (1/138500) * (3412/1)
        
        # Natural Gas: (dollars/cf) * (thousand cf/1000 cf) * (1 cf natural gas/1039 BTU) * (3412 BTU/1 kWh)
        elif row['fuel_type'] == 'naturalGas':
            df_fuelPrices_perkWh.at[index, f'{year}_fuelPrice_perkWh'] = row[f'{year}_nominal_unit_price'] * (1/1000) * (1/1039) * (3412/1)
        
        # Electricity: convert cents per kWh to $ per kWh
        elif row['fuel_type'] == 'electricity':
            df_fuelPrices_perkWh.at[index, f'{year}_fuelPrice_perkWh'] = row[f'{year}_nominal_unit_price'] / 100

# Convert nominal dollars to real 2021 US dollars (USD2021)
df_fuelPrices_perkWh['2018_fuelPrice_perkWh'] = df_fuelPrices_perkWh['2018_fuelPrice_perkWh'] * cpi_ratio_2021_2018
df_fuelPrices_perkWh['2019_fuelPrice_perkWh'] = df_fuelPrices_perkWh['2019_fuelPrice_perkWh'] * cpi_ratio_2021_2019
df_fuelPrices_perkWh['2020_fuelPrice_perkWh'] = df_fuelPrices_perkWh['2020_fuelPrice_perkWh'] * cpi_ratio_2021_2020
df_fuelPrices_perkWh['2021_fuelPrice_perkWh'] = df_fuelPrices_perkWh['2021_fuelPrice_perkWh'] * cpi_ratio_2021_2021
df_fuelPrices_perkWh['2022_fuelPrice_perkWh'] = df_fuelPrices_perkWh['2022_fuelPrice_perkWh'] * cpi_ratio_2021_2022

# Original dictionary mapping census divisions to states
map_states_census_divisions = {
    "New England": ["CT", "ME", "MA", "NH", "RI", "VT"],
    "Middle Atlantic": ["NJ", "NY", "PA"],
    "East North Central": ["IN", "IL", "MI", "OH", "WI"],
    "West North Central": ["IA", "KS", "MN", "MO", "NE", "ND", "SD"],
    "South Atlantic": ["DE", "DC", "FL", "GA", "MD", "NC", "SC", "VA", "WV"],
    "East South Central": ["AL", "KY", "MS", "TN"],
    "West South Central": ["AR", "LA", "OK", "TX"],
    "Mountain": ["AZ", "CO", "ID", "NM", "MT", "UT", "NV", "WY"],
    "Pacific": ["AK", "CA", "HI", "OR", "WA"]
}

# Reverse the mapping to create a state-to-census-division map
state_to_census_division = {}
for division, states in map_states_census_divisions.items():
    for state in states:
        state_to_census_division[state] = division

# Function to map location to census division
def map_location_to_census_division(location):
    if location in state_to_census_division:
        return state_to_census_division[location]
    return location

# Apply the function to map locations using .loc
df_fuelPrices_perkWh.loc[:, 'census_division'] = df_fuelPrices_perkWh['location_map'].apply(map_location_to_census_division)
df_fuelPrices_perkWh


-------------------------------------------------------------------------------------------------------
Private Perspective: Annual Energy Costs
-------------------------------------------------------------------------------------------------------
- Step 1: Obtain Level Energy Fuel Cost Data from the EIA
- Step 2: Calculate Annual Operating (Fuel) Costs
-------------------------------------------------------------------------------------------------------
      
-------------------------------------------------------------------------------------------------------
Step 1: Obtain Level Energy Fuel Cost Data from the EIA
-------------------------------------------------------------------------------------------------------
**Data Sources for Excel workbook containing state average Residential fuel cost for each fuel in 2018**
- EIA State Electricity Price: https://www.eia.gov/electricity/state/archive/2018/
- EIA Natural Gas Prices: https://www.eia.gov/dnav/ng/ng_pri_sum_dcu_SPA_a.htm


,state_region,location_map,source,fuel_type,base_units,2018_nominal_unit_price,2019_nominal_unit_price,2020_nominal_unit_price,2021_nominal_unit_price,2022_nominal_unit_price,units,2018_fuelPrice_perkWh,2019_fuelPrice_perkWh,2020_fuelPrice_perkWh,2021_fuelPrice_perkWh,2022_fuelPrice_perkWh,census_division
0,National,National,EIA Average Residential Price,naturalGas,"NOMINAL, dollars per cf",10.50,10.51,10.78,12.18,14.75,USD2021 per kWh,0.037209,0.036581,0.037064,0.039998,0.044849,National
1,AL,AL,EIA Average Residential Price,naturalGas,"NOMINAL, dollars per cf",15.22,15.63,16.01,15.72,17.36,USD2021 per kWh,0.053935,0.054402,0.055046,0.051623,0.052785,East South Central
2,AK,AK,EIA Average Residential Price,naturalGas,"NOMINAL, dollars per cf",10.99,11.11,11.14,10.94,11.08,USD2021 per kWh,0.038945,0.038670,0.038302,0.035926,0.033690,Pacific
3,AZ,AZ,EIA Average Residential Price,naturalGas,"NOMINAL, dollars per cf",15.35,13.46,13.10,15.52,17.81,USD2021 per kWh,0.054396,0.046849,0.045041,0.050967,0.054153,Mountain
4,AR,AR,EIA Average Residential Price,naturalGas,"NOMINAL, dollars per cf",11.77,11.05,12.28,12.55,16.78,USD2021 per kWh,0.041709,0.038461,0.042221,0.041213,0.051021,West South Central
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,National,National,"EIA Weekly Heating Oil and Propane Prices, Dat...",fuelOil,"NOMINAL, dollars per gallon",3.20,3.09,2.55,3.02,4.70,USD2021 per kWh,0.085069,0.080683,0.065772,0.074399,0.107207,National
112,New England,New England,"EIA Weekly Heating Oil and Propane Prices, Dat...",fuelOil,"NOMINAL, dollars per gallon",3.17,3.03,2.47,2.96,4.67,USD2021 per kWh,0.084272,0.079116,0.063708,0.072921,0.106523,New England
113,Central Atlantic,Middle Atlantic,"EIA Weekly Heating Oil and Propane Prices, Dat...",fuelOil,"NOMINAL, dollars per gallon",3.26,3.18,2.65,3.11,4.79,USD2021 per kWh,0.086664,0.083033,0.068351,0.076616,0.109260,Middle Atlantic
114,Lower Atlantic,South Atlantic,"EIA Weekly Heating Oil and Propane Prices, Dat...",fuelOil,"NOMINAL, dollars per gallon",2.87,2.80,2.45,2.91,4.44,USD2021 per kWh,0.076296,0.073111,0.063192,0.071689,0.101276,South Atlantic


In [29]:
filename = 'fuel_price_projection_factors.csv'
relative_path = os.path.join(r"fuel_prices", filename)
file_path = os.path.join(project_root, relative_path)
df_fuelPrices_projection_factors = pd.read_csv(file_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

# Convert the factors dataframe into a lookup dictionary
factor_dict = df_fuelPrices_projection_factors.set_index(['region', 'fuel_type']).to_dict('index')

# Apply the function to each row in the DataFrame
projected_prices_df = df_fuelPrices_perkWh.apply(lambda row: project_future_prices(row, factor_dict), axis=1)

# Concatenate the projected prices with the original DataFrame
df_fuelPrices_perkWh_projected = pd.concat([df_fuelPrices_perkWh, projected_prices_df], axis=1)

# Create Fuel Price Lookup
fuel_price_lookup = create_fuel_price_lookup(df_fuelPrices_perkWh_projected)
fuel_price_lookup

Retrieved data for filename: fuel_price_projection_factors.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\fuel_prices\fuel_price_projection_factors.csv


{'National': {'naturalGas': {2018: 0.037208757312705974,
   2019: 0.0365813487567895,
   2020: 0.03706386593739151,
   2021: 0.03999822906641001,
   2022: 0.04484879286017567,
   2023: 0.043817270624391634,
   2024: 0.04013966960985723,
   2025: 0.03825602030972985,
   2026: 0.0366863125596237,
   2027: 0.035744487909560016,
   2028: 0.03525115118809808,
   2029: 0.03543054635953878,
   2030: 0.03560994153097949,
   2031: 0.03596873187386089,
   2032: 0.03628267342388212,
   2033: 0.03677601014534405,
   2034: 0.03717964928108563,
   2035: 0.03753843962396704,
   2036: 0.03771783479540774,
   2037: 0.03794207875970862,
   2038: 0.0383457178954502,
   2039: 0.0383457178954502,
   2040: 0.03865965944547143,
   2041: 0.03897360099549266,
   2042: 0.03906329858121301,
   2043: 0.03901844978835284,
   2044: 0.03888390340977231,
   2045: 0.03906329858121301,
   2046: 0.03969118168125547,
   2047: 0.039646332888395294,
   2048: 0.03960148409553512,
   2049: 0.03960148409553512,
   2050: 0.039

### Step 2: Calculate Annual Operating (Fuel) Costs

### Baseline Fuel Cost: WHOLE-HOME

In [31]:
print("""
-------------------------------------------------------------------------------------------------------
Step 2: Calculate Annual Operating (Fuel) Costs
-------------------------------------------------------------------------------------------------------
- Create a mapping dictionary for fuel types
- Create new merge columns to ensure a proper match.
- Merge df_copy with df_fuel_prices to get fuel prices for electricity, natural gas, propane, and fuel oil
- Calculate the per kWh fuel costs for each fuel type and region
- Calculate the baseline fuel cost 
-------------------------------------------------------------------------------------------------------
""")
# df_euss_am_baseline_home = df_euss_am_baseline_home.copy()
# calculate_annual_fuelCost(df, fuel_price_lookup, lifetime)
df_euss_am_baseline_home = calculate_annual_fuelCost(df=df_euss_am_baseline_home,
                                                     fuel_price_lookup=fuel_price_lookup,
                                                     )
df_euss_am_baseline_home


-------------------------------------------------------------------------------------------------------
Step 2: Calculate Annual Operating (Fuel) Costs
-------------------------------------------------------------------------------------------------------
- Create a mapping dictionary for fuel types
- Create new merge columns to ensure a proper match.
- Merge df_copy with df_fuel_prices to get fuel prices for electricity, natural gas, propane, and fuel oil
- Calculate the per kWh fuel costs for each fuel type and region
- Calculate the baseline fuel cost 
-------------------------------------------------------------------------------------------------------

Calculating BASELINE (no retrofit) fuel costs from 2018 to 2033 for heating
Calculating BASELINE (no retrofit) fuel costs from 2018 to 2030 for waterHeating
Calculating BASELINE (no retrofit) fuel costs from 2018 to 2031 for clothesDrying
Calculating BASELINE (no retrofit) fuel costs from 2018 to 2033 for cooking


,bldg_id,square_footage,census_region,census_division,census_division_recs,building_america_climate_zone,cambium_GEA_region,state,city,county,...,2023_baseline_cooking_fuelCost,2024_baseline_cooking_fuelCost,2025_baseline_cooking_fuelCost,2026_baseline_cooking_fuelCost,2027_baseline_cooking_fuelCost,2028_baseline_cooking_fuelCost,2029_baseline_cooking_fuelCost,2030_baseline_cooking_fuelCost,2031_baseline_cooking_fuelCost,2032_baseline_cooking_fuelCost
2,239,1690.0,South,East South Central,East South Central,Hot-Humid,SRSOc,AL,Not in a census Place,G0100390,...,56.40,45.39,44.67,44.31,44.77,45.75,46.17,46.74,47.41,48.08
3,273,1690.0,South,East South Central,East South Central,Mixed-Humid,SRSOc,AL,In another census Place,G0100150,...,68.40,63.93,61.88,60.18,58.97,58.33,58.12,57.84,57.98,58.47
4,307,1220.0,South,East South Central,East South Central,Hot-Humid,SRSOc,AL,Not in a census Place,G0100850,...,68.40,63.93,61.88,60.18,58.97,58.33,58.12,57.84,57.98,58.47
5,409,1220.0,South,East South Central,East South Central,Hot-Humid,SRSOc,AL,Not in a census Place,G0100050,...,58.14,46.79,46.05,45.67,46.15,47.16,47.59,48.18,48.87,49.56
7,517,1220.0,South,East South Central,East South Central,Mixed-Humid,SRSOc,AL,In another census Place,G0101270,...,48.02,44.89,43.45,42.25,41.40,40.96,40.81,40.61,40.71,41.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548905,548109,1690.0,West,Mountain,Mountain North,Cold,RMPAc,WY,In another census Place,G5600050,...,40.48,39.90,39.11,38.61,38.28,37.86,37.90,38.19,38.32,38.61
548907,548226,2176.0,West,Mountain,Mountain North,Cold,RMPAc,WY,In another census Place,G5600050,...,40.48,39.90,39.11,38.61,38.28,37.86,37.90,38.19,38.32,38.61
548908,548228,1690.0,West,Mountain,Mountain North,Cold,RMPAc,WY,Not in a census Place,G5600010,...,40.48,39.90,39.11,38.61,38.28,37.86,37.90,38.19,38.32,38.61
548910,548417,885.0,West,Mountain,Mountain North,Cold,RMPAc,WY,Casper,G5600250,...,48.56,47.86,46.91,46.31,45.91,45.41,45.46,45.81,45.96,46.31


### Area Median Income Data Used to determine LMI Designation and IRA Rebates Eligibility/Amount

In [32]:
# Collect Area Median Income Data at PUMA-resolution
filename = "nhgis0002_ds239_20185_puma.csv"
relative_path = os.path.join(r"equity_data", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

df_county_medianIncome = pd.read_csv(file_path, encoding='ISO-8859-1')
df_county_medianIncome = df_county_medianIncome.drop(0)
df_county_medianIncome = df_county_medianIncome.reset_index(drop=True)

cols_interest = ['GISJOIN', 'STUSAB', 'STATE', 'PUMAA', 'GEOID', 'NAME_E', 'AJZAE001', 'AJZAM001']
df_county_medianIncome = df_county_medianIncome[cols_interest]
df_county_medianIncome = df_county_medianIncome.rename(columns={"GISJOIN": "gis_joinID_puma", "STUSAB": "state_abbrev", "STATE": "state", "PUMAA": "puma_code", "GEOID": "census_geoID", "NAME_E": "name_estimate", "AJZAE001": "median_income_USD2018", "AJZAM001": "median_income_USD2018_marginOfError"})
df_county_medianIncome

Retrieved data for filename: nhgis0002_ds239_20185_puma.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\equity_data\nhgis0002_ds239_20185_puma.csv




,gis_joinID_puma,state_abbrev,state,puma_code,census_geoID,name_estimate,median_income_USD2018,median_income_USD2018_marginOfError
0,G01000100,AL,Alabama,100,79500US0100100,"Lauderdale, Colbert, Franklin & Marion (Northe...",44779,1317
1,G01000200,AL,Alabama,200,79500US0100200,Limestone & Madison (Outer) Counties--Huntsvil...,64779,2670
2,G01000301,AL,Alabama,301,79500US0100301,Huntsville (North) & Madison (East) Cities PUM...,62399,1595
3,G01000302,AL,Alabama,302,79500US0100302,"Huntsville City (Central & South) PUMA, Alabama",52811,1814
4,G01000400,AL,Alabama,400,79500US0100400,"DeKalb & Jackson Counties PUMA, Alabama",39499,1447
...,...,...,...,...,...,...,...,...
2373,G72000902,PR,Puerto Rico,902,79500US7200902,"Trujillo Alto, Canóvanas, Loíza & Carolina (Su...",25262,972
2374,G72001001,PR,Puerto Rico,1001,79500US7201001,"Caguas Municipio PUMA, Puerto Rico",25006,1128
2375,G72001002,PR,Puerto Rico,1002,79500US7201002,"Gurabo, San Lorenzo, Juncos & Las Piedras Muni...",22391,888
2376,G72001101,PR,Puerto Rico,1101,79500US7201101,"Río Grande, Fajardo, Luquillo, Ceiba, Vieques ...",20621,815


# Model Runtime

In [33]:
# Get the current datetime again
end_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Calculate the elapsed time
elapsed_time = datetime.strptime(end_time, "%Y-%m-%d_%H-%M-%S") - datetime.strptime(start_time, "%Y-%m-%d_%H-%M-%S")

# Format the elapsed time
elapsed_seconds = elapsed_time.total_seconds()
elapsed_minutes = int(elapsed_seconds // 60)
elapsed_seconds = int(elapsed_seconds % 60)

# Print the elapsed time
print(f"The code took {elapsed_minutes} minutes and {elapsed_seconds} seconds to execute.")

The code took 2 minutes and 10 seconds to execute.
